#Install

In [ ]:
!pip install python-pptx

from google.colab import drive
import os
import pptx
from pptx.enum.shapes import MSO_SHAPE_TYPE

!pip install openai
import openai
import os
import time
import pandas as pd
!pip install --upgrade openai

In [ ]:
#your API key
os.environ["OPENAI_API_KEY"] = '[YOUR_API_KEY]'

#load ppt from the google drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/analyzer/input_ppt")

#Functions

In [ ]:
class ChatApp: #no demo, only template
    def __init__(self, input):
        # Setting the API key to use the OpenAI API
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.messages = input

    def chat(self, message):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0.7,
            top_p = 1,
        )
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})

        for i in range(2):
          self.messages.pop()

        return response["choices"][0]["message"]["content"]

In [ ]:
def make_prompt_format(system, user, assistant): #prompt demonstration
  #system
  input = [{"role": "system", "content": system}]
  if user != "":
    #user
    demo_user = []
    demo_user.append(user)
    #assistant
    demo_assistant = []
    demo_assistant.append(assistant)

    #make format for API
    demo = pd.DataFrame({'user': demo_user, 'assistant': demo_assistant})
    for prompt, completion in zip(demo['user'], demo['assistant']):
      message_dict = {"role": "user", "content": prompt}
      input.append(message_dict)
      message_dict = {"role": "assistant", "content": completion}
      input.append(message_dict)
    
  return input

#Main

##Load Data

In [ ]:
#load ppt file
ppt = pptx.Presentation('Architecture_Design.pptx')

#for full slide
full_slide = []
content_full = "Entire contents: "
for i, slide in enumerate(ppt.slides):
  content_full = content_full + f"  *slide {i}: "
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_full = content_full + shape.text
full_slide = content_full

# print("*"*10)
# print(full_slide)

#for each slide, keywords
each_slide = {} #dictionary form
for i, slide in enumerate(ppt.slides):
  content_each = ""
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_each = content_each + shape.text
  each_slide[f'{i}'] = content_each 

# print("*"*10)
# print(each_slide)

##Each Slide

In [ ]:
# Summarize each Powerpoint file.

In [ ]:
#prompt demonstration
each_system = "I want you to summarize the given content. I will provide you with the contents of a PowerPoint presentation, and you will provide me with a summary of the slide."
each_user = "Design flaws – a few examples What if the system does not handle multiple customers purchasing items simultaneously ?\n\t\uf0e0the shopping cart becomes common for both customers! \uf04c\n\nWhat if web server does too much processing for each transaction? \n\t\uf0e0capacity of the web server comes down drastically & the web server crashes when too many requests come within a small-time window!\n\nWhat if the security requirements are ignored ? \n\t\uf0e0hacker can manipulate the prices of items in the shopping cart \n\nWhat if the team misses to identify several generic components for which COTS are available ?\n\t\uf0e0the team develops everything in-house…\n\n\n"
each_assistant = "The PowerPoint presentation highlights several design flaws such as the shopping cart becoming common for multiple customers, the web server crashing due to excessive processing, hackers manipulating prices due to ignored security requirements, and developing everything in-house due to missed identification of available COTS components."
prompt_each = make_prompt_format(each_system, each_user, each_assistant)

#summarize each slide
result_each = {}
app_each = ChatApp(prompt_each)
for i in range(len(each_slide)):
  message = each_slide[f'{i}']
  res = app_each.chat(message)
  time.sleep(3) #API call restriction, 20 calls per a minute
  result_each[f'{i}'] = res

#print result
result_each